# Filtrer navn fra bøker

In [1]:
import dhlab.nbtext as nb
import dhlab.token_map as tm
from dhlab.module_update import update, css
import dhlab.graph_networkx_louvain as gnl
import json
import re
import sys
from time import time
import pandas as pd
%matplotlib inline
css()

In [2]:
from collections import Counter

update('token_map', overwrite=True)
from token_map import filter_names, names_from_corpus, names_from_excel, count_names_corpus

Updated file `/Users/yngvilb/Desktop/NER/token_map.py`

### Ekstra kommandoer fra `token_map`

* names_from_corpus(korpus)
* count_names_corpus(korpus, token_map)
* names_from_excel(excelfil)
    

### Gazetter

Gazettene ligger her, følger med repoet fra Github.

In [3]:
# Gazetter basert på navn i forfatterregisteret på nb.no
first_name = json.load(open("NER_gazetter_norsk/fornavn.json"))
last_name = json.load(open("NER_gazetter_norsk/etternavn.json"))

In [ ]:
# Gazetter basert på navn i forfatterregisteret på nb.no
first_name = json.load(open("NER-gazetter/first_names.json"))
last_name = json.load(open("NER-gazetter/last_names.json"))

In [ ]:
#first_names = list(set(first_names) - {'Oslo'})

### Korpus

Hent noen bøker...

In [4]:
korpus = nb.pure_urn("""
URN

 https://urn.nb.no/URN:NBN:no-nb_digibok_
 https://urn.nb.no/URN:NBN:no-nb_digibok_2015110908267
 https://urn.nb.no/URN:NBN:no-nb_digibok_
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009061903027
 ikke digitalisert
 https://urn.nb.no/URN:NBN:no-nb_digibok_
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009072800065
 https://urn.nb.no/URN:NBN:no-nb_digibok_2010063006072
 https://urn.nb.no/URN:NBN:no-nb_digibok_
 https://urn.nb.no/URN:NBN:no-nb_digibok_
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009042204078
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014010708163
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009072800062
 https://urn.nb.no/URN:NBN:no-nb_digibok_2006111501039
 https://urn.nb.no/URN:NBN:no-nb_digibok_2006111501043
 https://urn.nb.no/URN:NBN:no-nb_digibok_2006112001058
 https://urn.nb.no/URN:NBN:no-nb_digibok_2006111701045
 https://urn.nb.no/URN:NBN:no-nb_digibok_2006111601041
 https://urn.nb.no/URN:NBN:no-nb_digibok_2006113000001
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014121608238 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008070804046
 https://urn.nb.no/URN:NBN:no-nb_digibok_
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013073108136
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009051210001
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009051212001
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009051213001
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009041703007
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009020503016
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009032003002
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012041808082
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012100907130
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008040204004
 https://urn.nb.no/URN:NBN:no-nb_digibok_2006082200053 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008073100001
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008072304027
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008070201046
 https://urn.nb.no/URN:NBN:no-nb_digibok_2015111007004
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008040804176
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014082708025
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014102108092
 https://urn.nb.no/URN:NBN:no-nb_digibok_2015110507024
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014082508209
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008080500028
 https://urn.nb.no/URN:NBN:no-nb_digibok_
 https://urn.nb.no/URN:NBN:no-nb_digibok_2010053106106
 https://urn.nb.no/URN:NBN:no-nb_digibok_
 https://urn.nb.no/URN:NBN:no-nb_digibok_2006120500036
 https://urn.nb.no/URN:NBN:no-nb_digibok_
 https://urn.nb.no/URN:NBN:no-nb_digibok_2006121301027
 https://urn.nb.no/URN:NBN:no-nb_digibok_
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009042204098
 https://urn.nb.no/URN:NBN:no-nb_digibok_2016070748069
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012050708083
 https://urn.nb.no/URN:NBN:no-nb_digibok_2016041908011
 https://urn.nb.no/URN:NBN:no-nb_digibok_2016041548100
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008072304028
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008073100001
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009062204014
 https://urn.nb.no/URN:NBN:no-nb_digibok_2016040608162
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014080508168
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007011801005
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007011801004
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007021501117
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007010300018
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007010300045
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007010500022
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007011201040
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014082108108
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009062201011
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013110508162
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014080508169
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009041404093
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009061701068
 https://urn.nb.no/URN:NBN:no-nb_digibok_2010061806078
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013110508163
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008042300025
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008042800109
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009041704061
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009042200011
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007060504048
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007042301017
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007030201038
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008051904054
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007031501098
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007032801122
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007021901144
https://urn.nb.no/URN:NBN:no-nb_digibok_2008051404001
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012021608266 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2016103148510 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009041704038
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009060204081
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009052904033
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008051904146
 https://urn.nb.no/URN:NBN:no-nb_digibok_2015012008072 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009061504071 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008051904126
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013101708014
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013101708013
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007032101125
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009041704058 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009062204039
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007042401029
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014070306049
 https://urn.nb.no/URN:NBN:no-nb_digibok_2016092948064
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009041404091
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007052504014
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009042200012 
 https://urn.nb.no/URN:NBN:no-nb_digibok_
 https://urn.nb.no/URN:NBN:no-nb_digibok_2016101048159
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009061701063
 https://urn.nb.no/URN:NBN:no-nb_digibok_2011020106014
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007072300005
 https://urn.nb.no/URN:NBN:no-nb_digibok_2011120808007
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012030208136 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007071700077
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012053108037
 https://urn.nb.no/URN:NBN:no-nb_digibok_2011111608068
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014021807021
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013102407331
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012022008023
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012022208159
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007072601080
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007070500096
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007070201034
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008052701029
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007060501039
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007060700028
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007060701007
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008111904058
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013012205070
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013030406047
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013040907068
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012100907038
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013102407066
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007062200011
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014030308499
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007061804087
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013062706017
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012091805011 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012071905088
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008060304065
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007070304070
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007070900076
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008060300007
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008052304109
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013061706091
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014071508124
 https://urn.nb.no/URN:NBN:no-nb_digibok_2011121405002
 https://urn.nb.no/URN:NBN:no-nb_digibok_2016101048269
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014082708002
 https://urn.nb.no/URN:NBN:no-nb_digibok_
 https://urn.nb.no/URN:NBN:no-nb_digibok_
 https://urn.nb.no/URN:NBN:no-nb_digibok_
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014031108214
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012062505011
 https://urn.nb.no/URN:NBN:no-nb_digibok_2015072308349 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012012708020
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008070400042 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013102407067 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013070205080
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012070205032
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012070205030
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007091901004  
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013093006082 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2015060808049
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007112600058
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012101908136
 https://urn.nb.no/URN:NBN:no-nb_digibok_2011020308070
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012101908137
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012101908135
 https://urn.nb.no/URN:NBN:no-nb_digibok_2010121708024
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013011607286
 https://urn.nb.no/URN:NBN:no-nb_digibok_
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008010300041 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008030704064
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008031304026
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008030700046
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008031304021
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008081204044
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008020600003
 https://urn.nb.no/URN:NBN:no-nb_digibok_2015041008041
 https://urn.nb.no/URN:NBN:no-nb_digibok_2010100508090
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008031101012 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014061608252
 https://urn.nb.no/URN:NBN:no-nb_digibok_2010060203030 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2010072703001
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009102700026
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008080804146
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009011900032
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008092300090
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009012204024
 https://urn.nb.no/URN:NBN:no-nb_digibok_2010011100089
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009041404041
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009030204015
 https://urn.nb.no/URN:NBN:no-nb_digibok_2010122120006 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008120800099 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008112000056
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009012904103
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009012200018
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009062601017
 https://urn.nb.no/URN:NBN:no-nb_digibok_2011083106080
 https://urn.nb.no/URN:NBN:no-nb_digibok_2011070505043
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014120907542
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013071705100
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014111007502
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012062505101
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013042408089 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013011408109
 https://urn.nb.no/URN:NBN:no-nb_digibok_2015111608028 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012102907161
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014022706007 
 https://urn.nb.no/URN:NBN:no-nb_digibok_
ikke digitalisert
""")

In [5]:
len(korpus)

195

In [6]:
df = nb.frame(nb.metadata(korpus),('urn forfatter årstall tittel subtittel utgiver sjanger sesam språk'.split()))

In [7]:
# urn i korpuset som vi ikke har metadata på
[x for x in korpus if x not in [str(y) for y in df.urn]]

['2009051212001', '2009051213001', '2011020308070']

In [8]:
# Her trekker vi fra de bøkene vi ikke har data for i korpuset
korpus = [str(y) for y in df.urn]
len(korpus)

192

In [9]:
# Fordi korpuset er så stort deler vi det i mindre deler for endel av uttrekksjobben

step=10
korp = dict()

for e in range(0,len(korpus), step):
    korp[e] = korpus[e:e+step]

In [10]:
korp

{0: ['2015110908267',
  '2009061903027',
  '2009072800065',
  '2010063006072',
  '2009042204078',
  '2014010708163',
  '2009072800062',
  '2006111501039',
  '2006111501043',
  '2006112001058'],
 10: ['2006111701045',
  '2006111601041',
  '2006113000001',
  '2014121608238',
  '2008070804046',
  '2013073108136',
  '2009051210001',
  '2009041703007',
  '2009020503016',
  '2009032003002'],
 20: ['2012041808082',
  '2012100907130',
  '2008040204004',
  '2006082200053',
  '2008073100001',
  '2008072304027',
  '2008070201046',
  '2015111007004',
  '2008040804176',
  '2014082708025'],
 30: ['2014102108092',
  '2015110507024',
  '2014082508209',
  '2008080500028',
  '2010053106106',
  '2006120500036',
  '2006121301027',
  '2009042204098',
  '2016070748069',
  '2012050708083'],
 40: ['2016041908011',
  '2016041548100',
  '2008072304028',
  '2008073100001',
  '2009062204014',
  '2016040608162',
  '2014080508168',
  '2007011801005',
  '2007011801004',
  '2007021501117'],
 50: ['2007010300018',
  '

In [11]:
korpusnavn = dict()

for e in korp:
    nå = time()
    print (e)
    korpusnavn[e] = names_from_corpus(korp[e])
    print ('tid,', round(time()-nå))
    

0
tid, 9
10
tid, 11
20
tid, 4
30
tid, 6
40
tid, 4
50
tid, 5
60
tid, 7
70
tid, 5
80
tid, 6
90
tid, 5
100
tid, 6
110
tid, 6
120
tid, 6
130
tid, 7
140
tid, 7
150
tid, 8
160
tid, 9
170
tid, 9
180
tid, 9
190
tid, 1


In [12]:
# Ser på uttrekket for en av bolkene. Vi ser her at mange av ordene som ikke er navn:
korpusnavn[0]

[Counter({'Aand': 52,
          'Aar': 1946,
          'Aaret': 368,
          'Aarhundrede': 118,
          'Aasen': 31,
          'Akt': 53,
          'Alt': 73,
          'Anledning': 535,
          'Arbejde': 301,
          'Arbejder': 255,
          'Arne': 54,
          'Asbjørnsen': 83,
          'Begavelse': 17,
          'Begreb': 20,
          'Bergen': 1751,
          'Betydning': 166,
          'Bifald': 29,
          'Billede': 60,
          'Billeder': 107,
          'Bind': 281,
          'Bjørnson': 207,
          'Bjørnsons': 64,
          'Blade': 392,
          'Blik': 54,
          'Bog': 187,
          'Bogen': 142,
          'Bonden': 24,
          'Bondens': 17,
          'Chr': 615,
          'Christiania': 457,
          'Collett': 107,
          'Dage': 179,
          'Danmark': 556,
          'Del': 629,
          'Digt': 413,
          'Digte': 490,
          'Digter': 160,
          'Digtere': 106,
          'Digteren': 87,
          'Digterens': 51,
      

In [13]:
korpusnavn_agg = (Counter(),Counter(),Counter(),Counter())
for tokens in korpusnavn:
    for i in range(4):
        korpusnavn_agg[i].update(korpusnavn[tokens][i])

In [14]:
reserve = []

In [15]:
reserve = 'Klopstock Oehlenschlager Oehlenschlåger Hermione  Paars Catharine Leonarda Frederikke Schweitz'.split()

Filtrer og rensk opp navnelisten med `filter_names()` importert fra `token_map`. Variablene `first_names` og `last_names` slås sammen til en liste som en felles gazett.

In [16]:
filtered_names = filter_names(korpusnavn_agg, first_name + last_name + reserve)

In [39]:
# ved å se på det som ligger i removed, kan vi finne flere navn vi ønsker å legge til i reserve-lista

filtered_names['removed']

(Counter({'Aand': 407,
          'Aar': 4036,
          'Aaret': 673,
          'Aarhundrede': 240,
          'Akt': 138,
          'Alt': 396,
          'Anledning': 1167,
          'Arbejde': 512,
          'Arbejder': 372,
          'Begavelse': 17,
          'Begreb': 20,
          'Bergen': 7983,
          'Betydning': 298,
          'Bifald': 29,
          'Billede': 131,
          'Billeder': 373,
          'Bind': 496,
          'Blade': 695,
          'Blik': 444,
          'Chr': 2136,
          'Christiania': 2584,
          'Dage': 526,
          'Danmark': 6127,
          'Del': 1162,
          'Digt': 1225,
          'Digte': 2480,
          'Digter': 291,
          'Digtere': 163,
          'Digteren': 172,
          'Digterens': 86,
          'Digtervirksomhed': 16,
          'Digtning': 215,
          'Digtningen': 24,
          'Digtninger': 36,
          'Digtnings': 13,
          'Drama': 211,
          'Dæmring': 132,
          'Død': 931,
          'Efterligning':

In [ ]:
 # nb.frame(nb.frame(filtered_names['filtered']).transpose().sum(axis=1)).to_excel('navn_filter_JMH.xls')

In [17]:
# navnene vi vil gå videre med legger vi her i en variabel for enkelhets skyld

neo = filtered_names['filtered']

In [18]:
neo

(Counter({'Aasen': 1874,
          'Arne': 3065,
          'Asbjørnsen': 1961,
          'Bjørnson': 9355,
          'Bjørnsons': 2496,
          'Bog': 570,
          'Bogen': 271,
          'Bonden': 26,
          'Bondens': 19,
          'Collett': 3048,
          'Dramer': 17,
          'Ewald': 497,
          'Foss': 232,
          'Grund': 421,
          'Hansen': 2045,
          'Henrik': 7071,
          'Ibsen': 9384,
          'Inger': 1666,
          'Ivar': 1844,
          'Johan': 7484,
          'Landet': 596,
          'Landets': 247,
          'Liv': 2109,
          'Mangler': 13,
          'Munch': 2575,
          'Munchs': 331,
          'Olsen': 722,
          'Sagen': 503,
          'Solbakken': 229,
          'Steder': 260,
          'Stykket': 509,
          'Sverre': 1412,
          'Synnøve': 345,
          'Tone': 148,
          'Vinje': 3334,
          'Welhaven': 5395,
          'Welhavens': 1013,
          'Wergeland': 10254,
          'Wergelands': 2941,
   

In [19]:
# fjerner navn som bare forekommer som singel-navn (enten kun etternavn eller kun fornavn), da disse trolig ikke egentlig er navneforekomster

def filter_single(names):
    res = Counter()
    for x in names[0]:
        for y in names[1]:
            if x in y:
                res[x] = names[0][x]
    return res
    

In [20]:
neo_single = filter_single(neo)

In [21]:
# det viser seg at navnene som består av 4 navn stort sett bare er rot, vi fjerner dem derfor her
# når det står neo[3] er de ikke fjernet. Counter() på sisteplass fjerner dem.

neo_ny = (neo_single, neo[1], neo[2], neo[3])

In [22]:
# forsøk på en grammatikk som gjør at vi ikke får etternavn etterfulgt av fornavn. Noen navn er imidlertid både etternavn og fornavn, så dette løser ikke hele problemet

def filter_name_struct(names):
    res = (names[0],Counter(),Counter(),Counter())
    for i in range(1,4):
        for x in names[i]:
            well_formed = True
            for index in range(0,len(x) - 1):
                #print(x)
                if (x[index + 1] in first_name and x[index + 1] not in last_name) & (x[index] in last_name and x[index] not in first_name):
                    print(x)
                    well_formed = False
            if well_formed == True:
                res[i][x] = names[i][x]
    return res
    

In [23]:
neo_struct = filter_name_struct(neo_ny)
# printer navn som fjernes

('Major', 'Henrik')
('Foged', 'Nils')
('Strømsø', 'August')
('Foged', 'Ole')
('Ven', 'Johan')
('Aker', 'Ivar')
('Major', 'Oluf')
('Major', 'Christopher')
('Støren', 'Nils')
('Kongsberg', 'Carl')
('Moss', 'Henrik')
('Moss', 'Jens')
('Major', 'Th')
('Kongsberg', 'Hans')
('Skien', 'Peder')
('Aker', 'Ole')
('Aker', 'Hans')
('Major', 'Hans')
('Hesselberg', 'Gustav')
('Lier', 'Nils')
('Major', 'Lars')
('Major', 'Jens')
('Holmestrand', 'Hans')
('Hersleb', 'Kr')
('Moss', 'Ulrik')
('Ibsen', 'Henrik')
('Grimstad', 'Henrik')
('Ven', 'Henrik')
('Ven', 'Ole')
('Skien', 'Knud')
('Foged', 'Jens')
('Skien', 'Hans')
('Huset', 'Carl')
('Gran', 'Wilhelm')
('Lindeman', 'August')
('London', 'Carl')
('Kongsberg', 'Lars')
('Farsund', 'Hans')
('Molde', 'April')
('Molde', 'Niels')
('Tønsberg', 'Hans')
('Tønsberg', 'Ole')
('Kongsberg', 'Peder')
('Gaard', 'Ole')
('Nordland', 'Jens')
('Major', 'Ole')
('Foged', 'Søren')
('Horten', 'Hans')
('Ven', 'Rasmus')
('Moss', 'Hans')
('Gausdal', 'Ole')
('Major', 'Johan')
('T

('Tone', 'Birkeland', 'Gunvor')
('Birkeland', 'Gunvor', 'Risa')
('Gunvor', 'Risa', 'Karin')
('Risa', 'Karin', 'Beate')
('Maurits', 'Hansen', 'Maurits')
('Hansen', 'Maurits', 'Hansen')
('Marie', 'Wexelsen', 'Johanne')
('Saxegaard', 'Margit', 'Ravn')
('Welhaven', 'Henrik', 'Wergeland')
('Bjørnstjerne', 'Bjørnson', 'Henrik')
('Bjørnson', 'Henrik', 'Ibsen')
('Henrik', 'Ibsen', 'Johan')
('Arnulf', 'Øverland', 'Knut')
('Øverland', 'Knut', 'Hamsun')
('Johnsen', 'Trond', 'Berg')
('Berg', 'Eriksen', 'Oluf')
('Eriksen', 'Oluf', 'Ryghs')
('Dahl', 'Eilert', 'Sundt')
('Arne', 'Garborg', 'Aasmund')
('Garborg', 'Aasmund', 'Olavsson')
('Jakobsen', 'Fridtjof', 'Nansens')
('Evensmo', 'Olav', 'Dalgard')
('Aksel', 'Selmer', 'Aksel')
('Selmer', 'Aksel', 'Selmer')
('Bjørnson', 'Bjørnstjerne', 'Bjørnson')
('Nordland', 'Knut', 'Hamsun')
('Kristofer', 'Uppdal', 'Kristofer')
('Uppdal', 'Kristofer', 'Uppdal')
('Cora', 'Sandel', 'Cora')
('Sandel', 'Cora', 'Sandel')
('Torborg', 'Nedreaas', 'Torborg')
('Nedreaas', 

In [ ]:
ny_tm = tm.token_map(neo_struct)

In [ ]:
ny_tm

In [25]:
nb.save_frame_to_excel(nb.frame(ny_tm), 'tokenmap.xls', index=True)

In [23]:
nb.frame(ny_tm)

,0,1
0,"(Aleksander,)","(Ole, Aleksander)"
1,"(Alf,)","(Alf, Prøysen)"
2,"(Alnæs,)","(Finn, Alnæs)"
3,"(Anders,)","(Sigurd, Hoels, Anders)"
4,"(Andersen,)","(Anne, Marie, Andersen)"
...,...,...
777,"(Saabye, Christensens, Kim)","(Saabye, Christensens, Kim, Karlsen)"
778,"(Christensens, Kim, Karlsen)","(Saabye, Christensens, Kim, Karlsen)"
779,"(Bjørg, Øygarden, Øygarden)","(Bjørg, Øygarden, Øygarden)"
780,"(Kjell, Ivar, Andersen)","(Kjell, Ivar, Andersen)"


In [24]:
tm.show_names(neo_struct)

Lag 1

    Wergeland - 10254
    Ibsen - 9384
    Bjørnson - 9355
    Olav - 7543
    Johan - 7484
    Henrik - 7071
    Hans - 6613
    Holberg - 6256
    Oslo - 6238
    Welhaven - 5395
    Sigurd - 5118
    Garborg - 3995
    Lie - 3770
    Snorre - 3707
    Hamsun - 3560
    Vinje - 3334
    Jonas - 3235
    Kielland - 3129
    Moe - 3076
    Ibsens - 3067
    Arne - 3065
    Island - 3050
    Collett - 3048
    Bull - 3023
    Christian - 2984
    Wergelands - 2941
    Harald - 2902
    Maj - 2884
    Per - 2737
    August - 2703
    Gunnar - 2690
    Ole - 2598
    Munch - 2575
    Bin - 2527
    Bjørnsons - 2496
    Nils - 2439
    Peer - 2425
    April - 2343
    Jens - 2338
    Håkon - 2319
    Carl - 2280
    Peter - 2279
    Vesaas - 2230
    Knut - 2202
    Peder - 2180
    Liv - 2109
    Camilla - 2085
    Erik - 2057
    Hansen - 2045
    Ludvig - 2041
    Jørgen - 2016
    Sigrid - 2004
    Magnus - 1965
    Asbjørnsen - 1961
    Brandes - 1961
    Heiberg - 1944
    Dag

    Boye - 60
    Grøndal - 60
    Elvestad - 60
    Hagens - 60
    Thore - 60
    Blix - 59
    Oluf - 59
    Lippe - 59
    Colban - 59
    Jølsens - 59
    Herje - 59
    Bøge - 59
    Volds - 59
    Øklands - 59
    Karsten - 59
    Brochmann - 59
    Bern - 58
    Catharine - 58
    Vilhelmine - 58
    Johans - 58
    Bogens - 58
    Sørensen - 58
    Solumsmoen - 58
    Magnusson - 58
    Ella - 58
    Øivind - 58
    Joachim - 57
    Ross - 57
    Louis - 57
    Thiis - 57
    Skjæraasen - 57
    Strømsø - 56
    Caspari - 56
    Schjøtt - 56
    Kjølstad - 56
    Breda - 56
    Haukland - 56
    Clara - 56
    Hjertenæs - 56
    Boeck - 55
    Schreuder - 55
    Sem - 55
    Nag - 55
    Ørjasæters - 55
    Tollefsen - 55
    Grepstad - 55
    Foyn - 55
    Emma - 54
    Hjort - 54
    Lyngar - 54
    Osmundsen - 54
    Scheen - 54
    Lier - 53
    Løvdahl - 53
    Øberg - 53
    Erna - 53
    Engvald - 53
    Røsbak - 53
    Oftedal - 52
    Riis - 52
    Lyder - 52
    Stab

    Fosby - 12
    Nustad - 12
    Backes - 12
    Persen - 12
    Aamodt - 12
    Bramness - 12
    Helleve - 12
    Hertervig - 12
    Langås - 12
    Lindell - 12
    Rand - 12
    Schultz - 12
    Supphellen - 12
    Werenskiold - 12
    Wilse - 12
    Børsum - 12
    Mohn - 12
    Hermann - 12
    Ewos - 12
    Grete - 12
    Rønnaug - 12
    Svingen - 12
    Arnestad - 12
    Lommerud - 12
    Martinus - 12
    Norheim - 12
    Stensrud - 12
    Sørli - 12
    Wasberg - 12
    Myre - 12
    Fritzner - 11
    Marine - 11
    Paludan - 11
    Borger - 11
    Sigholt - 11
    Wiese - 11
    Christofer - 11
    Vandvik - 11
    Buch - 11
    Hildur - 11
    Coucheron - 11
    Larsens - 11
    Eiken - 11
    Skogstad - 11
    Cold - 11
    Hove - 11
    Andresen - 11
    Butenschøn - 11
    Nanna - 11
    Wollstonecraft - 11
    Drude - 11
    Heber - 11
    Lilly - 11
    Øbergs - 11
    Asta - 11
    Eugenie - 11
    Graah - 11
    Kaia - 11
    Takvam - 11
    Alver - 11
    Diesen

    Hanne Ørstavik - 18
    Petter Næss - 18
    Clara Tschudi - 18
    Jens Edvard - 17
    Johanne Margrethe - 17
    Johan Georg - 17
    Inger Charlotte - 17
    Erik Pontoppidans - 17
    Simon Kildal - 17
    Herman Foss - 17
    Ole Gabriel - 17
    Johan Welhaven - 17
    Hans Gram - 17
    Olav Aukrusts - 17
    Egil Rasmussen - 17
    Undsets Kristin - 17
    Olav Bø - 17
    Nordahl Rolf - 17
    Fredrik Engelstad - 17
    Asbjørn Aarseth - 17
    Knut Gjengedal - 17
    Thorbjørn Egners - 17
    Hobæk Haff - 17
    Harald Bakke - 17
    Otlu Alsvik - 17
    Erik Volds - 17
    Evi Bøgenæs - 17
    Anka Borch - 17
    Berg Eriksen - 17
    Svein Nyhus - 17
    Kim Karlsen - 17
    Harald Eeg - 17
    Olaf Hansen - 16
    Dorothea Maria - 16
    Wilhelm Andreas - 16
    Emil Stang - 16
    Claus Berg - 16
    Erik Bøgh - 16
    Ivar Åsen - 16
    Ludwig Tieck - 16
    Jacob Munch - 16
    Christian Boeck - 16
    Hans Strøm - 16
    Louis Kvalstad - 16
    Jørgen Bukdahl - 16

    Marie Margrethe - 6
    Lars Christian - 6
    Fredrik Ludvig - 6
    Marie Lovise - 6
    Anna Johanne - 6
    Elisabeth Christine - 6
    Marie Hansen - 6
    Christian Peter - 6
    Carl Peter - 6
    Margrethe Marie - 6
    Magdalene Margrethe - 6
    Carl Nicolai - 6
    Andreas Christian - 6
    Elise Sophie - 6
    Sophie Marie - 6
    Jørgen Andreas - 6
    Marie Caroline - 6
    Kristian Jensen - 6
    Ingeborg Christine - 6
    Jens Christopher - 6
    Frederik Anton - 6
    Christian Henrik - 6
    Karen Elisabeth - 6
    Henrik Anton - 6
    Christian Holst - 6
    Gabriel Kielland - 6
    Christian Ulrik - 6
    Carl Andreas - 6
    Christian Vilhelm - 6
    Jens Larsen - 6
    Christopher Henrik - 6
    Ludvig Mathias - 6
    Sophie Charlotte - 6
    Peter Treschow - 6
    Ole Peter - 6
    Bernt Andreas - 6
    Andreas Jensen - 6
    Sophie Magdalene - 6
    Jens Johan - 6
    Jakob Christian - 6
    Kristian Fredrik - 6
    Morten Luther - 6
    Ludvig Meyer - 6
   

...

    Ingeborg Jensen - 1
    Caroline Olsen - 1
    Birgitte Lund - 1
    Eker Jørgen - 1
    Oscar Frederik - 1
    Christiane Louise - 1
    Margrethe Aall - 1
    Inger Andrea - 1
    Andrea Riis - 1
    Maria Birgitte - 1
    Marius Peter - 1
    Elisabeth Paludan - 1
    Karen Roll - 1
    Niels Rosing - 1
    Georg Kristian - 1
    John Olaf - 1
    Herman Vang - 1
    Caroline Louise - 1
    Anna Henriette - 1
    Foged Anders - 1
    Frederikke Kirstine - 1
    Kirstine Holst - 1
    Hans Søren - 1
    Marie Nicoline - 1
    Hedevig Nicoline - 1
    Elisabeth Ross - 1
    Claus Thomas - 1
    Grimstad Frederik - 1
    Olaf Wilhelm - 1
    Elisabeth Brandt - 1
    Marius Pedersen - 1
    Sophie Fredrikke - 1
    Karen Hedevig - 1
    Camilla Wilhelmine - 1
    Jacob Meyer - 1
    Axel Jacob - 1
    Carl Magnus - 1
    Hans Hus - 1
    Cathrine Neumann - 1
    Nicolai Stoud - 1
    Else Catharine - 1
    Cathrine Stoud - 1
    Carl Stoud - 1
    Oscar Ludvig - 1
    Sophie Munk - 

    Laura Amalie - 1
    Elisabeth Wilhelmine - 1
    Andreas Oscar - 1
    Johan Clausen - 1
    Clausen Dahl - 1
    Niels Frederik - 1
    Wilhelmine Nielsen - 1
    Olaus W.s - 1
    Wolff Wolff - 1
    Niels August - 1
    Karen Wetlesen - 1
    Lars Johnsen - 1
    Cathrine Elise - 1
    Martha Olsen - 1
    Augusta Stenersen - 1
    Dorothea Aall - 1
    Jacobine Christine - 1
    Ludvig Peter - 1
    Carl Oscar - 1
    Margrethe Wulfsberg - 1
    Gustav Oscar - 1
    Elise Wulfsberg - 1
    Holmestrand Fredrik - 1
    Elisabeth Blom - 1
    Elise Amalie - 1
    Holt Nicolai - 1
    Christine Andre - 1
    Vilhelm Falck - 1
    Vilhelmine Nielsen - 1
    Maria Lund - 1
    Christine Magnus - 1
    Anders Johnsen - 1
    Maria Falck - 1
    Ola Vik - 1
    Gerhard Andreas - 1
    Birgitte Moe - 1
    Margrethe Johanne - 1
    Johannes Bergh - 1
    Andreas Kraft - 1
    Andreas Nielsen - 1
    Fredrik Vinsnes - 1
    Aars Minde - 1
    Martha Kirstine - 1
    Johannes Steenstrup 

    Andreas Eriksen - 1
    Larsen Arvid - 1
    Liv Anne - 1
    Ola Aleksander - 1
    Hans Martin - 1
    Leif Isaksen - 1
    Marie Winsnes - 1
    Anne Østby - 1
    Tryggve Anker - 1
    Odd Eriksen - 1
    Berit Evensmo - 1
    Sigurd Faldbakken - 1
    Odd Hovden - 1
    Berit Larsen - 1
    Ove Pedersen - 1
    Kåre Rud - 1
    Lars Alnæs - 1
    Roar Eriksen - 1
    Håkon Fangen - 1
    Jørgen Hagen - 1
    Øyvind Hougen - 1
    Barbra Rud - 1
    Johan Røsbak - 1
    Ole Østby - 1
    Stange Ansteinsson - 1
    Kristin Michaelsen - 1
    Groven Moen - 1
    Helge Stensrud - 1
    Magne Thesen - 1
    Helge Høiby - 1
    Vigdis Stubberud - 1
    Juel Sørli - 1
    Olaf Juel - 1
    Ellen Stein - 1
    Erling Tangen - 1
    Terje Øygarden - 1
    Jon Fredheim - 1
    Ludvig Hoel - 1
    Sigurd Kjølstad - 1
    Eidem Moe - 1
    Hans Lommerud - 1
    Magnus Hamsun - 1
    Arne Isaksen - 1
    Kristian Larsen - 1
    Karin Lie - 1
    Martinus Lund - 1
    Oskar Fossum - 1
    L

    Anna Sophie Muller - 1
    Marie Charlotte Petersen - 1
    Sophie Margrethe Knudsen - 1
    Ida Marie Louise - 1
    Christine Aall Lange - 1
    Maren Marie Holst - 1
    Elisabeth Maria Lasson - 1
    Johan Albert Julius - 1
    Foged Peter Christian - 1
    Anne Birgitte Marie - 1
    Birgitte Marie Kr - 1
    Johanne Marie Lie - 1
    Henriette Caroline Lovise - 1
    Anne Marie Johannesen - 1
    Hans Henrik Olsen - 1
    Harald Christian Georg - 1
    Konow Koren Konow - 1
    Christiane Johanne Molde - 1
    Helene Johanne Jacobsen - 1
    Carl Fredrik August - 1
    Otto Carl Martin - 1
    Helene Sophie Meyer - 1
    Birgitte Marie Lasson - 1
    Christine Sophie Petersen - 1
    Dorothea Christine Helene - 1
    Caroline Christiane Krogh - 1
    Georg Anton Krogh - 1
    Anne Helene Nicoline - 1
    Nils Jakob Jensen - 1
    Anne Maria Stub - 1
    Karen Cathrine Kirstine - 1
    Anne Nicoline Aall - 1
    Laura Christine Larsen - 1
    Henriette Sophie Amalie - 1
    Gr

    Rud Nils Johan - 1
    Henrik Ibsen Ibsen - 1
    Mauritz Christopher Hansen - 1
    Gerhard Gran Peter - 1
    Gran Peter Egge - 1
    Gabriel Scott Johan - 1
    Arnulf Øverland Øverland - 1
    Kielland Jonas Lie - 1
    Johan Falkberget Falkberget - 1
    Schanche Herman Wildenvey - 1
    Johan Nordahl Rolf - 1
    Johan Nordahl Rolfsen - 1
    Per Sivles Lars - 1
    Svend Holst Jensen - 1
    Johanne Vogt Lie - 1
    Jens Lund Andersen - 1
    Anders Christensen Hagen - 1
    Peder Paars Holberg - 1
    Rolf Egil Moe - 1
    Hans Alexander Hansen - 1
    Egil Nygaard Brekke - 1
    Nils Magne Knudsen - 1
    Ole Rønning Johannesen - 1
    Gunnar Odd Larsen - 1
    Zinken Hopp Aschehougs - 1
    Strøm Aas Moren - 1
    Synnøve Solbakken Bjørnson - 1
    Collett Vinje Welhaven - 1
    Vinje Welhaven Wergeland - 1
    Wessel Holberg Petter - 1
    Holberg Petter Dass - 1
    Alexander Kiellands Jacob - 1
    Inger Marie Andersen - 1
    Nordahl Griegs Ung - 1
    Johan Ruds Min 

In [35]:
opptelling = count_names_corpus(korpus, ny_tm)

In [39]:
len(opptelling)

214

In [40]:
## Gazettene inneholder feil med for- og etternavn
## vi har problem med genitiv

sjekk = lambda x: {'last': x in last_name, 'first': x in first_name}

In [41]:
sjekk('Tønsberg')

{'last': True, 'first': False}

In [ ]:
sum_opptelling = nb.frame_sort(nb.frame(opptelling_agg.sum(axis=1)))
sum_opptelling

In [ ]:
nb.save_frame_to_excel(sum_opptelling, 'sum_opptelling.xls', index=True)

Generer navn med nb.names()

In [ ]:
korpusnavn = names_from_corpus(korpus)

Se på names med tm.show_names().

In [ ]:
tm.show_names(korpusnavn)

`filtered_names` er en dict med to attributt, ett `filtered` og ett `removed`, sånn at man kan se hva som er fjernet og hvordan det filtrerte funkerer. Om det er fjernet for mye kan det bare legges til i gazetten og hele filtreringen kan kjøres på nytt. Merk at 4-ordsnavnene er bedre, men det mangler noe fremdeles på formen. Litt navnegrammatikk kunne gjøre susen her. Det er greit å få til. Nedenfor ser vi at det blir feil med 4-ordsnavn på formen "fornavn fornavn etternavn fornavn" som kan lukes vekk med en grammatikk.

Lagre de filtrerte navnene til en excelfil.

In [ ]:
nb.frame(nb.frame(filtered_names['filtered']).transpose().sum(axis=1)).to_excel('navn_filter.xls')

Hent de redigerte navnene tilbake. Det kan legges til navn og slettes navn fra filen. 
Endre filnavn i tilfelle lagring fra Excel har endret på det.

In [ ]:
neo_names = names_from_excel('navn_filter.xls')


Det tar litt tid å beregne `token_map` så greit å legge den i variabel

In [ ]:
neo_token_map = tm.token_map(neo_names)


### Data fra boken

Tell opp navn med `tm.count_name_strings` ... det ser ut som det fremdeles trengs til litt opprydding. Spesielt de lange navne lager litt ballade, men det er mulig å fjerne dem helt. Nettverk under der igjen. Bøker som ikke er med i grunndataene vil generere en melding om feil.

In [ ]:
opptelling = count_names_corpus(korpus, neo_token_map)

In [ ]:
opptelling[:100]